In [52]:
from fonctions.gestion_bdd import get_data_bdd, lire_bdd_perso, requete_perso_bdd
from time import sleep
from fonctions.match import (matchlol,
                             get_summoner_by_puuid
                             
                             )
from tqdm import tqdm
from aiohttp import ClientSession
import pandas as pd
import traceback
import sys


In [53]:
df = lire_bdd_perso('''SELECT joueur, match_id from matchs where season = 13''', index_col='joueur').T
df

,match_id
joueur,
rektexo,EUW1_6543336385
bacquante,EUW1_6543451648
rektexo,EUW1_6543451648
rektexo,EUW1_6543484825
bacquante,EUW1_6543395943
...,...
aigrikiki,EUW1_6559786208
aigrikiki,EUW1_6559896016
aigrikiki,EUW1_6559958424


In [54]:
df_joueur = lire_bdd_perso('''SELECT tracker.index, tracker.puuid
                            from tracker ''').T
df_joueur

,puuid
index,
linò,EdwvX1ztDHKS7CQEujeZQs9HKla93F81JALpKgYCMiRbLk...
aigrikiki,Cm8otDqPlNun7dxgtQK8Bg98ePGK59CjgBTVw3zEE9z4Xr...
chguizou,4BTE8cIQ-wDahe4rWYgr3aS9ATI-rliTXPTSlZURSPivCj...
bacquante,73lncLh1uW1t1dWqvbb4wQrcU2UXGmIz_4nA6nRjF45P_g...
exorblue,IrLpMUK7XCjVXr39Opqxeu_1srF7gOs6-iRquaPGvLTZN2...
kulbutoké,OvxfDc4pYVDiJE5YsvXRhdT1gZtrE2sR6nUsCKN-4gPnjY...
rektexo,fSdIMo_hHNAgamgKThGjd_SDyWd9QaCMteLFdwc8BgsJ23...
xinamal,5cYweJze1ApZV-1ZCRPIV3smqfIe5ljLRHYXRvfKGQGblb...
ylarabka,Q7kqEUEhRQqwLUxDxKGLXv_IyiCxGGHZj-PLsGJAepV5ys...


In [55]:
session = ClientSession()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000002111AE8D4E0>


In [56]:
dict_me = {}
for joueur in df.index.unique():
    me = await get_summoner_by_puuid(df_joueur.loc[joueur, 'puuid'], session)
    dict_me[joueur] = me
    
dict_me['raphifanboy'] = dict_me['aigrikiki']
dict_me['kikiienclaket'] = dict_me['aigrikiki']
dict_me['dontbenpcplz'] = dict_me['aigrikiki']
dict_me['Stαte'] = dict_me['dadstate']

In [57]:
df_error = pd.DataFrame(columns=['match_id'])

In [58]:
data_cut = df

for joueur, data in tqdm(data_cut.iterrows(), total=len(data_cut)):
    
    puuid = df_joueur.loc[joueur, 'puuid']

    if joueur == 'aigrikiki':
        
        for joueur_selected in ['aigrikiki', 'raphifanboy', 'kikiienclaket', 'dontbenpcplz']:
            try:
                match_info = matchlol(joueur_selected,
                                        idgames=0,
                                        identifiant_game=data['match_id'],
                                        me=dict_me[joueur_selected])
                await match_info.get_data_riot()

            
                await match_info.prepare_data()
                break
            except KeyError:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                traceback_details = traceback.format_exception(exc_type, exc_value, exc_traceback)
                traceback_msg = ''.join(traceback_details)
                print(traceback_msg)
                print(data['match_id'])
                continue
    
    elif joueur == 'dadstate':
        
        for joueur_selected in ['Stαte', 'dadstate']:   
            
            try:
                match_info = matchlol(joueur_selected,
                                        idgames=0,
                                        identifiant_game=data['match_id'],
                                        me=dict_me[joueur_selected])
                await match_info.get_data_riot()

            
                await match_info.prepare_data()
                break
            except KeyError:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                traceback_details = traceback.format_exception(exc_type, exc_value, exc_traceback)
                traceback_msg = ''.join(traceback_details)
                print(traceback_msg)
                print(data['match_id'])
                continue 
        
    else:

        try:
            match_info = matchlol(joueur,
                                    idgames=0,
                                    identifiant_game=data['match_id'],
                                    me=dict_me[joueur])
            await match_info.get_data_riot()

        
            await match_info.prepare_data()
        except KeyError:
            df_error.loc[data['match_id'], 'match_id'] = joueur
            continue

                    
       
    requete_perso_bdd('''UPDATE matchs SET gold_share = :gold_share, 
                        ecart_gold_team = :ecart_gold_team,
                        temps_avant_premiere_mort = :temps_avant_premiere_mort,
                        ecart_gold = :ecart_gold
                        WHERE match_id = :match_id AND joueur = :joueur''', 
                        {'gold_share': match_info.gold_share,
                        'ecart_gold_team': match_info.ecart_gold_team,
                        'temps_avant_premiere_mort': match_info.thisTimeLiving,
                        'ecart_gold': match_info.ecart_gold_noformat,
                        'match_id': data['match_id'],
                        'joueur': joueur})
        
    sleep(10)
    



In [59]:
# df_error.to_csv('error2.csv')